# Lab | Unsupervised learning intro

### Instructions

--------------------------------------------------------------------------------------------------
It's the moment to perform clustering on the songs you collected. Remember that the ultimate goal of this little project is to improve the recommendations of artists. Clustering the songs will allow the recommendation system to limit the scope of the recommendations to only songs that belong to the same cluster - songs with similar audio features.

The experiments you did with the Spotify API and the Billboard web scraping will allow you to create a pipeline such that when the user enters a song, you:

1. Check whether or not the song is in the Billboard Hot 200.
2. Collect the audio features from the Spotify API.

After that, you want to send the Spotify audio features of the submitted song to the clustering model, which should return a cluster number.

We want to have as many songs as possible to create the clustering model, so we will add the songs you collected to a bigger dataset available on Kaggle containing 160 thousand songs.

--------------------------------------------------------------------------------------------------------

In [15]:
!pip install spotipy

In [16]:
# Testing spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="c43ae4f18c0d4b2c8b04c93649fa4b72",
                                                           client_secret="1f7865f1e7aa439e9e997bc38b591855"))

In [13]:
import numpy as np
import pandas as pd

# Importing the track features csv file
spotify_track_features = pd.read_csv(r"C:\Users\mafal\Documents\ironhack\labs\lab-unsupervised-learning-intro\spotify_track_features.csv")
spotify_track_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.695,0.700,1,-1.587,0,0.0332,0.104000,0.000000,0.192,0.461,94.959,audio_features,2ikmBwZKZr0ahGcX4x8qtj,spotify:track:2ikmBwZKZr0ahGcX4x8qtj,https://api.spotify.com/v1/tracks/2ikmBwZKZr0a...,https://api.spotify.com/v1/audio-analysis/2ikm...,183296,4
1,0.593,0.741,4,-4.353,0,0.0359,0.022100,0.000000,0.393,0.460,96.978,audio_features,2MxErftY5S07dFtIdxQOSF,spotify:track:2MxErftY5S07dFtIdxQOSF,https://api.spotify.com/v1/tracks/2MxErftY5S07...,https://api.spotify.com/v1/audio-analysis/2MxE...,220670,4
2,0.660,0.765,2,-6.217,1,0.0299,0.125000,0.000956,0.235,0.681,123.051,audio_features,19meO0ADnoTjRuBMXZCdbs,spotify:track:19meO0ADnoTjRuBMXZCdbs,https://api.spotify.com/v1/tracks/19meO0ADnoTj...,https://api.spotify.com/v1/audio-analysis/19me...,175333,4
3,0.577,0.891,0,-4.672,1,0.0359,0.001230,0.000000,0.114,0.846,144.989,audio_features,3OPyobYAM5MgTm35AJV99O,spotify:track:3OPyobYAM5MgTm35AJV99O,https://api.spotify.com/v1/tracks/3OPyobYAM5Mg...,https://api.spotify.com/v1/audio-analysis/3OPy...,155707,4
4,0.531,0.693,6,-5.203,0,0.0374,0.009310,0.000003,0.119,0.555,157.960,audio_features,4nDfJDZaUVtwOSnGROb2GN,spotify:track:4nDfJDZaUVtwOSnGROb2GN,https://api.spotify.com/v1/tracks/4nDfJDZaUVtw...,https://api.spotify.com/v1/audio-analysis/4nDf...,164453,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3003,0.406,0.807,7,-3.871,1,0.0507,0.008350,0.000000,0.118,0.290,159.713,audio_features,38ODYA4I5jEhFr4xJJd1RG,spotify:track:38ODYA4I5jEhFr4xJJd1RG,https://api.spotify.com/v1/tracks/38ODYA4I5jEh...,https://api.spotify.com/v1/audio-analysis/38OD...,226861,3
3004,0.502,0.961,9,-4.389,1,0.0905,0.000075,0.000002,0.124,0.281,110.028,audio_features,3oGNDHK33fp1GMqU9e4HQ7,spotify:track:3oGNDHK33fp1GMqU9e4HQ7,https://api.spotify.com/v1/tracks/3oGNDHK33fp1...,https://api.spotify.com/v1/audio-analysis/3oGN...,234550,4
3005,0.639,0.832,1,-4.976,1,0.1180,0.003080,0.000382,0.121,0.482,119.045,audio_features,2jPqRiw1kJvxDKIibCPhHu,spotify:track:2jPqRiw1kJvxDKIibCPhHu,https://api.spotify.com/v1/tracks/2jPqRiw1kJvx...,https://api.spotify.com/v1/audio-analysis/2jPq...,166467,4
3006,0.741,0.810,11,-5.808,0,0.1650,0.002650,0.018400,0.131,0.799,132.076,audio_features,0mH0iiNINYULYFwszeqWnW,spotify:track:0mH0iiNINYULYFwszeqWnW,https://api.spotify.com/v1/tracks/0mH0iiNINYUL...,https://api.spotify.com/v1/audio-analysis/0mH0...,125455,4


In [24]:
# Checking what columns does the track features df contains
spotify_track_features.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [23]:
# Importing the track data csv file
spotify_track_data = pd.read_csv(r"C:\Users\mafal\Documents\ironhack\labs\lab-unsupervised-learning-intro\spotify_track_data.csv")
spotify_track_data

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,...,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,198123,False,https://api.spotify.com/v1/tracks/7hgIaQykdol1...,7hgIaQykdol1sWnj1uqBup,False,CULT CLASSIC,31,...,3vw9IZ3YV2T5bqYTYG0IXr,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Cult Classic,2023-11-10,day,6,album,spotify:album:3vw9IZ3YV2T5bqYTYG0IXr,QMRSZ2302147,https://open.spotify.com/track/7hgIaQykdol1sWn...
1,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,188808,False,https://api.spotify.com/v1/tracks/4pYiIZn2DKrK...,4pYiIZn2DKrK8MBYuS946R,False,BACKSTABBER,31,...,1RFijpHOUqGPZ3hdvcm8IM,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",BACKSTABBER,2024-01-05,day,1,album,spotify:album:1RFijpHOUqGPZ3hdvcm8IM,QZWFH2374284,https://open.spotify.com/track/4pYiIZn2DKrK8MB...
2,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,195226,False,https://api.spotify.com/v1/tracks/3xeyZGEEVW8S...,3xeyZGEEVW8SZPmteR9Fw6,False,Love Goes On,31,...,30eJbXVjjtYtYtwkt337Sr,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Love Goes On,2023-05-12,day,1,album,spotify:album:30eJbXVjjtYtYtwkt337Sr,QZQAY2345784,https://open.spotify.com/track/3xeyZGEEVW8SZPm...
3,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,93251,False,https://api.spotify.com/v1/tracks/1mj4y7NHlTq6...,1mj4y7NHlTq6YfTZOeNqOx,False,Tornillo,31,...,6MSMis63C7wWLVSSmSl92b,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Tornillo,2023-11-16,day,1,album,spotify:album:6MSMis63C7wWLVSSmSl92b,USHR22316801,https://open.spotify.com/track/1mj4y7NHlTq6YfT...
4,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,155706,True,https://api.spotify.com/v1/tracks/3OPyobYAM5Mg...,3OPyobYAM5MgTm35AJV99O,False,you don't like me like that,31,...,35YQAprIaYWKCWtr7iS5UT,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",you don't like me like that,2023-05-19,day,2,album,spotify:album:35YQAprIaYWKCWtr7iS5UT,USHR22316102,https://open.spotify.com/track/3OPyobYAM5MgTm3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3003,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,199906,True,https://api.spotify.com/v1/tracks/5W8YXBz9MTID...,5W8YXBz9MTIDyrpYaCg2Ky,False,Last Resort,81,...,0BHa0ePkvGAVKymB4FU58m,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Infest,2001-04-25,day,13,album,spotify:album:0BHa0ePkvGAVKymB4FU58m,USDW10021712,https://open.spotify.com/track/5W8YXBz9MTIDyrp...
3004,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,210240,False,https://api.spotify.com/v1/tracks/2DlHlPMa4M17...,2DlHlPMa4M17kufBvI2lEN,False,Chop Suey!,85,...,6jWde94ln40epKIQCd8XUh,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Toxicity,2001-09-04,day,15,album,spotify:album:6jWde94ln40epKIQCd8XUh,USSM10107256,https://open.spotify.com/track/2DlHlPMa4M17kuf...
3005,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CL...",1,157333,False,https://api.spotify.com/v1/tracks/3K4HG9evC7dg...,3K4HG9evC7dg3N0R9cYqk4,False,One Step Closer,82,...,6hPkbAV3ZXpGZBGUvL6jVM,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Hybrid Theory (Bonus Edition),2000,year,15,album,spotify:album:6hPkbAV3ZXpGZBGUvL6jVM,USWB10002399,https://open.spotify.com/track/3K4HG9evC7dg3N0...
3006,[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,225306,True,https://api.spotify.com/v1/tracks/6nJPHXRpKYv2...,6nJPHXRpKYv2yqtalEjK

In [25]:
# Checking what columns does the track data df contains
spotify_track_data.columns

Index(['artists', 'available_markets', 'disc_number', 'duration_ms',
       'explicit', 'href', 'id', 'is_local', 'name', 'popularity',
       'preview_url', 'track_number', 'type', 'uri', 'album.album_type',
       'album.artists', 'album.available_markets',
       'album.external_urls.spotify', 'album.href', 'album.id', 'album.images',
       'album.name', 'album.release_date', 'album.release_date_precision',
       'album.total_tracks', 'album.type', 'album.uri', 'external_ids.isrc',
       'external_urls.spotify'],
      dtype='object')

In [29]:
spotify_track_data[['id', 'name', 'album.artists']]

,id,name,album.artists
0,7hgIaQykdol1sWnj1uqBup,CULT CLASSIC,[{'external_urls': {'spotify': 'https://open.s...
1,4pYiIZn2DKrK8MBYuS946R,BACKSTABBER,[{'external_urls': {'spotify': 'https://open.s...
2,3xeyZGEEVW8SZPmteR9Fw6,Love Goes On,[{'external_urls': {'spotify': 'https://open.s...
3,1mj4y7NHlTq6YfTZOeNqOx,Tornillo,[{'external_urls': {'spotify': 'https://open.s...
4,3OPyobYAM5MgTm35AJV99O,you don't like me like that,[{'external_urls': {'spotify': 'https://open.s...
...,...,...,...
3003,5W8YXBz9MTIDyrpYaCg2Ky,Last Resort,[{'external_urls': {'spotify': 'https://open.s...
3004,2DlHlPMa4M17kufBvI2lEN,Chop Suey!,[{'external_urls': {'spotify': 'https://open.s...
3005,3K4HG9evC7dg3N0R9cYqk4,One Step Closer,[{'external_urls': {'spotify': 'https://open.s...
3006,6nJPHXRpKYv2yqtalEjKy5,Got the Life,[{'external_urls': {'spotify': 'https://open.s...


In [37]:
spotify_track_data.iloc[0]['album.artists']

"[{'external_urls': {'spotify': 'https://open.spotify.com/artist/2dTOWcCL0cYviin0Uz1lj4'}, 'href': 'https://api.spotify.com/v1/artists/2dTOWcCL0cYviin0Uz1lj4', 'id': '2dTOWcCL0cYviin0Uz1lj4', 'name': 'Holy Wars', 'type': 'artist', 'uri': 'spotify:artist:2dTOWcCL0cYviin0Uz1lj4'}]"

In [39]:
import ast  # Import the Abstract Syntax Trees module

# Function that extracts the artist name from the column album.artist
def extract_artist_name(artist_album_str):
    artist_album_list = ast.literal_eval(artist_album_str)
    artist_name = artist_album_list[0]['name']
    return artist_name

# Adding artist_name column to our track data df
spotify_track_data['artist_name'] = spotify_track_data['album.artists'].apply(extract_artist_name)
spotify_track_data[['id', 'name', 'album.artists', 'artist_name']]

,id,name,album.artists,artist_name
0,7hgIaQykdol1sWnj1uqBup,CULT CLASSIC,[{'external_urls': {'spotify': 'https://open.s...,Holy Wars
1,4pYiIZn2DKrK8MBYuS946R,BACKSTABBER,[{'external_urls': {'spotify': 'https://open.s...,"Ergo, Bria"
2,3xeyZGEEVW8SZPmteR9Fw6,Love Goes On,[{'external_urls': {'spotify': 'https://open.s...,Kelsy Karter & The Heroines
3,1mj4y7NHlTq6YfTZOeNqOx,Tornillo,[{'external_urls': {'spotify': 'https://open.s...,Margaritas Podridas
4,3OPyobYAM5MgTm35AJV99O,you don't like me like that,[{'external_urls': {'spotify': 'https://open.s...,Zeph
...,...,...,...,...
3003,5W8YXBz9MTIDyrpYaCg2Ky,Last Resort,[{'external_urls': {'spotify': 'https://open.s...,Papa Roach
3004,2DlHlPMa4M17kufBvI2lEN,Chop Suey!,[{'external_urls': {'spotify': 'https://open.s...,System Of A Down
3005,3K4HG9evC7dg3N0R9cYqk4,One Step Closer,[{'external_urls': {'spotify': 'https://open.s...,Linkin Park
3006,6nJPHXRpKYv2yqtalEjKy5,Got the Life,[{'external_urls': {'spotify': 'https://open.s...,Korn


In [54]:
# Function that extracts the artist id from the column album.artist
def extract_artist_id(artist_album_str):
    artist_album_list = ast.literal_eval(artist_album_str)
    artist_id = artist_album_list[0]['id']
    return artist_id

# Adding artist_id column to our track data df
spotify_track_data['artist_id'] = spotify_track_data['album.artists'].apply(extract_artist_id)
spotify_track_data[['id', 'name', 'album.artists', 'artist_name', 'artist_id']]

,id,name,album.artists,artist_name,artist_id
0,7hgIaQykdol1sWnj1uqBup,CULT CLASSIC,[{'external_urls': {'spotify': 'https://open.s...,Holy Wars,2dTOWcCL0cYviin0Uz1lj4
1,4pYiIZn2DKrK8MBYuS946R,BACKSTABBER,[{'external_urls': {'spotify': 'https://open.s...,"Ergo, Bria",0AF9HrL08aOaZPsIiO8GmA
2,3xeyZGEEVW8SZPmteR9Fw6,Love Goes On,[{'external_urls': {'spotify': 'https://open.s...,Kelsy Karter & The Heroines,2mAAO54PkHr3NjdlRpzEDl
3,1mj4y7NHlTq6YfTZOeNqOx,Tornillo,[{'external_urls': {'spotify': 'https://open.s...,Margaritas Podridas,5O9NicFLG2F9Xr7OHxmrb7
4,3OPyobYAM5MgTm35AJV99O,you don't like me like that,[{'external_urls': {'spotify': 'https://open.s...,Zeph,502gYHkFCtLzBIcU4ctPLd
...,...,...,...,...,...
3003,5W8YXBz9MTIDyrpYaCg2Ky,Last Resort,[{'external_urls': {'spotify': 'https://open.s...,Papa Roach,4RddZ3iHvSpGV4dvATac9X
3004,2DlHlPMa4M17kufBvI2lEN,Chop Suey!,[{'external_urls': {'spotify': 'https://open.s...,System Of A Down,5eAWCfyUhZtHHtBdNk56l1
3005,3K4HG9evC7dg3N0R9cYqk4,One Step Closer,[{'external_urls': {'spotify': 'https://open.s...,Linkin Park,6XyY86QOPPrYVGvF9ch6wz
3006,6nJPHXRpKYv2yqtalEjKy5,Got the Life,[{'external_urls': {'spotify': 'https://open.s...,Korn,3RNrq3jvMZxD9ZyoOZbQOD


In [ ]:
# Function that gets the artist's genre
def get_artist_genre(artist_id):
    return sp.artist(artist_id)['genres']

# Adding artist_id column to our track data df
spotify_track_data['artist_genre'] = spotify_track_data['artist_id'].apply(get_artist_genre)
spotify_track_data[['id', 'name', 'album.artists', 'artist_name', 'artist_id', 'artist_genre']]

------------------------------------------------------------------------------------------------
We need to get the audio features from the songs we extract on the Web Scraping lab.

----------------------------------------------------------------------------------------------

In [49]:
results = sp.search(q='artist:System of a Down, Chop Suey!', limit=32, type='track')
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3ASystem+of+a+Down%2C+Chop+Suey%21&type=track&offset=0&limit=32',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5eAWCfyUhZtHHtBdNk56l1'},
       'href': 'https://api.spotify.com/v1/artists/5eAWCfyUhZtHHtBdNk56l1',
       'id': '5eAWCfyUhZtHHtBdNk56l1',
       'name': 'System Of A Down',
       'type': 'artist',
       'uri': 'spotify:artist:5eAWCfyUhZtHHtBdNk56l1'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'HU',
      'IS',
      'IE',
      'IT',
      'LV',
      'LT',
      'LU',
      'MY',
      'MT',
      'MX',
      'NL',
      'NZ',
      'NI',
     

In [45]:
sp.audio_features('17phhZDn6oGtzMe56NuWvj')

[{'danceability': 0.561,
  'energy': 0.604,
  'key': 9,
  'loudness': -4.409,
  'mode': 1,
  'speechiness': 0.0337,
  'acousticness': 0.199,
  'instrumentalness': 1.9e-05,
  'liveness': 0.104,
  'valence': 0.242,
  'tempo': 159.92,
  'type': 'audio_features',
  'id': '17phhZDn6oGtzMe56NuWvj',
  'uri': 'spotify:track:17phhZDn6oGtzMe56NuWvj',
  'track_href': 'https://api.spotify.com/v1/tracks/17phhZDn6oGtzMe56NuWvj',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/17phhZDn6oGtzMe56NuWvj',
  'duration_ms': 210689,
  'time_signature': 3}]

In [46]:
sp.track('17phhZDn6oGtzMe56NuWvj')

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/33qOK5uJ8AR2xuQQAhHump'},
    'href': 'https://api.spotify.com/v1/artists/33qOK5uJ8AR2xuQQAhHump',
    'id': '33qOK5uJ8AR2xuQQAhHump',
    'name': 'Teddy Swims',
    'type': 'artist',
    'uri': 'spotify:artist:33qOK5uJ8AR2xuQQAhHump'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'US',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',
   'OM',


In [56]:
sp.artist('5eAWCfyUhZtHHtBdNk56l1')#['genres']

{'external_urls': {'spotify': 'https://open.spotify.com/artist/5eAWCfyUhZtHHtBdNk56l1'},
 'followers': {'href': None, 'total': 10301459},
 'genres': ['alternative metal', 'nu metal', 'rap metal', 'rock'],
 'href': 'https://api.spotify.com/v1/artists/5eAWCfyUhZtHHtBdNk56l1',
 'id': '5eAWCfyUhZtHHtBdNk56l1',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb60063d3451ade8f9fab397c2',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab6761610000517460063d3451ade8f9fab397c2',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f17860063d3451ade8f9fab397c2',
   'width': 160}],
 'name': 'System Of A Down',
 'popularity': 80,
 'type': 'artist',
 'uri': 'spotify:artist:5eAWCfyUhZtHHtBdNk56l1'}

###### 1. Creating the clusters

---------------------------------------------------------------------------------------------
We'll first start by creating the cluster using the track features.

----------------------------------------------------------------------------------------------------

In [ ]:
X = spotify_track_features[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'duration_ms', 'time_signature']].copy()